# Libraries

In [1]:
!python -V # --version

Python 3.12.1


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

import pickle # To be able to save the model

In [ ]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiment') # Set experiment name. MLflow will create it if it doesn't exist and assigns all runs to it.

2025/05/23 19:11:13 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp-2025/02-experiment-tracking/mlruns/1', creation_time=1748027473246, experiment_id='1', last_update_time=1748027473246, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
# Output:
# Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.
# <Experiment: artifact_location='/workspaces/mlops-zoomcamp-2025/02-experiment-tracking/mlruns/1', creation_time=1748027473246, experiment_id='1', last_update_time=1748027473246, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [23]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

# hyperopt - library that uses Bayesian optimisation to find the best hyperparameters
# fmin - method to minimise the objective function
# tpe - Tree-structured Parzen Estimator, a Bayesian optimisation algorithm
# hp - hyperopt library for defining the search space
# STATUS_OK - status of the trial
# Trials - class to keep track of the trials
# scope - used to define the search space for hyperparameters

# Data

Use the following commands to download the data locally: 
- `mkdir /workspaces/mlops-zoomcamp-2025/01-intro/data`
  
- `cd 01-intro/data`
  
- `wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet` - download data for January, 2021

- `wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet` - download data for February, 2021

In [6]:
%%time 

path_to_data = './data/green_tripdata_2021-01.parquet'
df = pd.read_parquet(path_to_data)

CPU times: user 42.6 ms, sys: 18.1 ms, total: 60.7 ms
Wall time: 54.7 ms


In [7]:
# Calculate target variable: trip duration
df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']

# Convert duration to minutes
df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)

# Check percentage of data within 1 and 60 minute durations
trip_duration_1_to_60_condition = (df['duration'] >= 1) & (df['duration'] <= 60)

# Filter the data to only include trips with duration between 1 and 60 minutes
df = df.loc[trip_duration_1_to_60_condition]

# Feature Engineering

In [8]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [ ]:
# Use one-hot encoding to convert categorical columns to numerical
# One-hot encoding converts strings into categorical variables. Convert int64 to string first

df[categorical] = df[categorical].astype(str)

In [10]:
%%time 

train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

# Target variable
target = 'duration'
y_train = df[target].values

CPU times: user 266 ms, sys: 15.9 ms, total: 282 ms
Wall time: 291 ms


# Linear Regression

In [11]:
%%time 

lr = LinearRegression()
lr.fit(X_train, y_train)

# Make predictions
y_pred = lr.predict(X_train)

# Evaluate the performance of the model: use RMSE 
root_mean_squared_error(y_train, y_pred)

CPU times: user 195 ms, sys: 1.97 ms, total: 197 ms
Wall time: 31.8 ms


9.827368941909368

# Function to pre-process the data

In [12]:
def read_dataframe(filename):
    # Read in the data
    df = pd.read_parquet(filename)

    # Calculate target variable: trip duration
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']

    # Convert duration to minutes
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)

    # Check percentage of data within 1 and 60 minute durations
    trip_duration_1_to_60_condition = (df['duration'] >= 1) & (df['duration'] <= 60)
    df = df.loc[trip_duration_1_to_60_condition]

    # Feature engineering
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [13]:
%%time 

df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

CPU times: user 455 ms, sys: 47.9 ms, total: 503 ms
Wall time: 468 ms


In [14]:
# Check the size of dataframes
len(df_train), len(df_val)

(73908, 61921)

In [15]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts) # Notice, that we only transform the validation set, not fitting required

In [16]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [17]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

10.496651120492572

# LASSO Regression

In [18]:
# Default parameters of Lasso
lr = Lasso()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

12.212583224318818

In [19]:
# Change default parameters of Lasso
lr = Lasso(alpha=0.001)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

10.481255832596219

# Keeping track of runs with MLflow

In [21]:
with mlflow.start_run(): # Start a new run. Everything below will be associated with this run
    # Start logging information
    mlflow.set_tag('developer', 'timur')

    mlflow.log_param('train-data-path', './data/green_tripdata_2021-01.parquet') # Any parameters relevant to the model
    mlflow.log_param('valid-data-path', './data/green_tripdata_2021-02.parquet') 

    
    alpha = 0.1
    mlflow.log_param('alpha', alpha)
    lr = Lasso(alpha=alpha)
    lr.fit(X_train, y_train)    
    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)

    mlflow.log_metric('rmse', rmse)

# Hyperparameter Tuning with MLflow

In [26]:
# To use XGBost, we need to convert the data into DMatrix format
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [24]:
# Define the objective function
def objective(params):
    # params - set of hyperparameters for XGBoost

    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
# Define the search space for hyperparameters
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # values between exp(-3) and exp(0), -0.05 and 1
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear', # reg:linear is now deprecated in favor of reg:squarederror
    'seed': 42
}

In [ ]:
# Pass the information about objective function and search space to fmin
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest, # Alogorithm to use for hyperparameter tuning
    max_evals=50,
    trials=Trials(),
    verbose=False # Set to True to see the progress of the trials
)

# Train the model with the best hyperparameters

In [28]:
# Based on the results of the 50 runs, get the model with the lowest RMSE

params = {
    "learning_rate": 0.14539214740033007,
    "max_depth": 11,
    "min_child_weight": 3.4888218672829265,
    "objective": "reg:linear",
    "reg_alpha": 0.012807793708352775,
    "reg_lambda": 0.25871874053728533,
    "seed": 42
}

# Use autologging to log the parameters. This simplifies the process of logging parameters and metrics
mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

2025/05/23 20:23:42 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.4.2 <= xgboost <= 3.0.0, but the installed version is 3.0.1. If you encounter errors during autologging, try upgrading / downgrading xgboost to a compatible version, or try upgrading MLflow.
2025/05/23 20:23:42 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '89947b2c3e9b42499a6ab382256a4f44', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:11.06559
[1]	validation-rmse:10.13502
[2]	validation-rmse:9.38665
[3]	validation-rmse:8.78767
[4]	validation-rmse:8.30662
[5]	validation-rmse:7.92792
[6]	validation-rmse:7.63116


/workspaces/mlops-zoomcamp-2025/venvs/mlops/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [20:23:43] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[7]	validation-rmse:7.39648
[8]	validation-rmse:7.21185
[9]	validation-rmse:7.06555
[10]	validation-rmse:6.94930
[11]	validation-rmse:6.85803
[12]	validation-rmse:6.78439
[13]	validation-rmse:6.72441
[14]	validation-rmse:6.67701
[15]	validation-rmse:6.63710
[16]	validation-rmse:6.60459
[17]	validation-rmse:6.57596
[18]	validation-rmse:6.55295
[19]	validation-rmse:6.53178
[20]	validation-rmse:6.51358
[21]	validation-rmse:6.49698
[22]	validation-rmse:6.48501
[23]	validation-rmse:6.47471
[24]	validation-rmse:6.46294
[25]	validation-rmse:6.45325
[26]	validation-rmse:6.44399
[27]	validation-rmse:6.43476
[28]	validation-rmse:6.42930
[29]	validation-rmse:6.42158
[30]	validation-rmse:6.41581
[31]	validation-rmse:6.40956
[32]	validation-rmse:6.40459
[33]	validation-rmse:6.39820
[34]	validation-rmse:6.39564
[35]	validation-rmse:6.39000
[36]	validation-rmse:6.38642
[37]	validation-rmse:6.37872
[38]	validation-rmse:6.37539
[39]	validation-rmse:6.37372
[40]	validation-rmse:6.36905
[41]	validation-r

2025/05/23 20:23:49 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/05/23 20:23:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/workspaces/mlops-zoomcamp-2025/venvs/mlops/lib/python3.12/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [20:23:49] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/05/23 20:23:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


# Saving the Model

In [127]:
%%time 

# Save the dictionary vectoriser and selected model
# mode wb - write binary
with open('./models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

CPU times: user 7.09 ms, sys: 0 ns, total: 7.09 ms
Wall time: 15.2 ms
